In [1]:
import numpy as np

In [19]:
pos = np.array([[0,0], [0, 1], [1, 0], [1, 1]])
pos_x = pos[:, 0, None]
pos_y = pos[:, 1, None]

In [3]:
# Constants

boxSize = 1 # cm
nbins = 3

In [6]:
s_pos = boxSize / nbins / 2.
e_pos = boxSize - boxSize / nbins / 2
bins = np.linspace(s_pos, e_pos, nbins)
bins

array([0.16666667, 0.5       , 0.83333333])

In [7]:
pos

array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])

In [31]:
x_idx = np.argmin(np.abs(pos_x - bins), axis=1)
y_idx = np.argmin(np.abs(pos_y - bins), axis=1)
x_idx, y_idx

(array([0, 0, 2, 2]), array([0, 2, 0, 2]))

In [38]:
bin_idx = np.ravel_multi_index((y_idx, x_idx), (nbins, nbins))

In [40]:
bin_idx

array([0, 6, 2, 8])

In [52]:
posgrid = np.eye(nbins**2)[bin_idx]


In [47]:
import seaborn as sns

In [51]:
pos

array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])

In [55]:
posgrid

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.]])

## Testing 

In [1]:
import scipy.io
import numpy as np

In [2]:
data = scipy.io.loadmat('data_for_cell77.mat')

In [3]:
pos_x = data['posx_c'].flatten()
pos_y = data['posy_c'].flatten()


In [4]:
boxSize = 100 # cm
nbins = 20

In [5]:
bins = np.linspace(boxSize / nbins / 2, boxSize - boxSize / nbins / 2, nbins)

In [6]:
x_indices = np.abs(pos_x[:, None] - bins).argmin(axis=1)
y_indices = np.abs(pos_y[:, None] - bins).argmin(axis=1)

In [11]:
bin_indices = (nbins - 1 - y_indices) * nbins + x_indices

In [16]:
pos = np.column_stack((pos_x, pos_y))
posgrid = np.zeros((len(pos), nbins * nbins), dtype=int)
posgrid[np.arange(len(pos)), bin_indices] = 1

In [17]:
posgrid

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(145150, 400))

In [18]:
data_compare = scipy.io.loadmat('pos.mat')
posgrid_compare = data_compare['posgrid']

In [19]:
if np.any(posgrid - posgrid_compare):
    print("Non zero elements present")

Non zero elements present


In [17]:
posgrid

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(145150, 400))

In [16]:
posgrid_compare

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(145150, 400), dtype=uint8)

In [20]:
comparison = np.array_equal(posgrid, posgrid_compare)
comparison

False

In [23]:
difference = np.where(posgrid != posgrid_compare)
print("Mismatched indices:", difference)
print("Python values:", posgrid[difference])
print("MATLAB values:", posgrid_compare[difference])

Mismatched indices: (array([     0,      0,      1, ..., 145148, 145149, 145149],
      shape=(273886,)), array([  3,  60,   3, ..., 381,  39, 381], shape=(273886,)))
Python values: [0 1 0 ... 1 0 1]
MATLAB values: [1 0 1 ... 0 1 0]


In [49]:
import numpy as np

def pos_map_python(pos, nbins, box_size):
    """
    Python equivalent of MATLAB's pos_map function with Python-style 0-based indexing.
    
    Parameters:
        pos (numpy.ndarray): Nx2 array with x and y coordinates.
        nbins (int): Number of bins per dimension.
        box_size (float): Size of the box.

    Returns:
        posgrid (numpy.ndarray): Binary matrix representing positional bins.
        bins (numpy.ndarray): Bin centers.
    """
    # Define bin centers similar to MATLAB code
    bins = np.linspace(box_size / nbins / 2, box_size - box_size / nbins / 2, nbins)

    # Initialize the position grid
    posgrid = np.zeros((len(pos), nbins**2), dtype=int)

    # Loop over positions to assign bin indices
    for idx in range(len(pos)):
        xcoor = np.argmin(np.abs(pos[idx, 0] - bins))
        ycoor = np.argmin(np.abs(pos[idx, 1] - bins))

        # Convert to zero-based index
        bin_idx = (nbins - ycoor - 1) * nbins + xcoor  

        posgrid[idx, bin_idx] = 1  # Assign position in posgrid

    return posgrid, bins

# Example usage with loaded data
import scipy.io

# Load the MATLAB file
file_path = "data_for_cell77.mat"
mat_data = scipy.io.loadmat(file_path)

# Extract relevant data
pos = np.hstack((mat_data['posx_c'], mat_data['posy_c']))
nbins = 20
box_size = mat_data['boxSize'][0, 0]

# Compute posgrid using Python function
posgrid_python, bins_python = pos_map_python(pos, nbins, box_size)


In [50]:
posgrid_python

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(145150, 400))

In [51]:
bins

array([ 2.5,  7.5, 12.5, 17.5, 22.5, 27.5, 32.5, 37.5, 42.5, 47.5, 52.5,
       57.5, 62.5, 67.5, 72.5, 77.5, 82.5, 87.5, 92.5, 97.5])

In [52]:
pos_matlab = scipy.io.loadmat('posgrid_matlab.mat')

In [53]:
posgrid_matlab = pos_matlab['posgrid']

In [54]:
np.array_equal(posgrid_python, posgrid_matlab)

False

In [55]:
difference = np.where(posgrid_python != posgrid_matlab)
difference


(array([     0,      0,      1, ..., 145148, 145149, 145149],
       shape=(273886,)),
 array([  3,  60,   3, ..., 381,  39, 381], shape=(273886,)))

In [56]:
print("Mismatched indices:", difference)
print("Python values:", posgrid_python[difference])
print("MATLAB values:", posgrid_matlab[difference])

Mismatched indices: (array([     0,      0,      1, ..., 145148, 145149, 145149],
      shape=(273886,)), array([  3,  60,   3, ..., 381,  39, 381], shape=(273886,)))
Python values: [0 1 0 ... 1 0 1]
MATLAB values: [1 0 1 ... 0 1 0]


In [57]:
len(posgrid_python[difference])

273886

In [58]:
len(posgrid_matlab[difference])

273886